# Absolute Zero at -272°C, says J. H. Lambert in 1777

In his book ["Pyrometrie"](https://ia600909.us.archive.org/7/items/bub_gb_G5I_AAAAcAAJ/bub_gb_G5I_AAAAcAAJ.pdf), page 40, Lambert states measurements by Mr. Amonton concerning the temperature and pressure of air. On page 47, he improves on the temperature of boiling water. By extrapolation, he re-calculates the temperature of air at pressure zero. Today's value of this absolute zero is -273,15°C. So his -272°C is pretty close.

## Code Preamble

In [1]:
(require '[clojupyter.misc.helper :as helper])
(def _ (helper/add-dependencies '[org.scicloj/noj "2-beta4"]))

#'user/_

In [2]:
(ns lambert
  (:require [scicloj.kindly.v4.kind :as kind]
            [clojure.math :as math]))

nil

In [3]:
(defn transpose [m]
  (apply mapv vector m))

(defn tenpow [n]
  (apply * (repeat n 10)))

(defn round [x dec]
  (double (/ (math/round (* x (tenpow dec))) (tenpow dec))))

#'lambert/round

## Data and Calculation

The book is in German. So, "Zolle" means inches, that is pressure; "Grade" means degrees, temperature. He was kind enough to give verbal descriptions, "frierendes Wasser" is frozen water, i.e. our 0°C, and "siedendes Wasser" is boiling water, i.e. our 100°C.

In [4]:
(def desc {:zolle "Zolle des Amontonschen Thermometers"
           :grade "Grade das Luftthermometers"})

#'lambert/desc

In [5]:
(def data
  {:zolle [51.6 54 54.4 58.2 58.9 55.9 59.9 61.10 64.2 67.3 58.5 73.0 54.0]
   :grade [1000 1049 1055 1129 1141 1082 1160 1201 1246 1306 1134 1417 1049]
   :C     ["frierendes Wasser"
           "Im Keller der Pariser Sternwarte"
           "Sternwarte2"
           "Wärem der Hand in mässiger Sommerluft, von verschiedenen Personen"
           "Handwärme 2"
           "Wärme der Luft, zur Zeit, da diese Wärme der Hände beobachtet worden"
           "schmelzende Butter"
           "schmelzend Unschlit"
           "gerinnend Wachs"
           "ein 10 bis 12 Gran schweres Stückchen Wachses ist ganz geschmolzen"
           "temperiert scheinendes Wasser"
           "siedendes Wasser"
           "temperierte Luft"]})

#'lambert/data

In [6]:
(def tb1 (transpose ((juxt :zolle :grade :C) data)))

#'lambert/tb1

Convert his degrees to Celsius, make sure that 0° and 100° are correctly attributed. Remember that these are Amonton's values.

In [7]:
(defn luft->celsius [g siedepunkt]
  (double (/ (- g 1000) (/ (- siedepunkt 1000) 100))))

(defn tb2 [sp]
  (mapv (fn[[z g d]] [z g (luft->celsius g sp) d]) tb1))

(tb2 1417)

51.6 1000 0.0 frierendes Wasser 54 1049 11.75059952038369 Im Keller der Pariser Sternwarte 54.4 1055 13.189448441247 Sternwarte2 58.2 1129 30.93525179856115 Wärem der Hand in mässiger Sommerluft, von verschiedenen Personen 58.9 1141 33.81294964028777 Handwärme 2 55.9 1082 19.66426858513189 Wärme der Luft, zur Zeit, da diese Wärme der Hände beobachtet worden 59.9 1160 38.36930455635492 schmelzende Butter 61.1 1201 48.20143884892086 schmelzend Unschlit 64.2 1246 58.99280575539568 gerinnend Wachs 67.3 1306 73.38129496402878 ein 10 bis 12 Gran schweres Stückchen Wachses ist ganz geschmolzen 58.5 1134 32.13429256594724 temperiert scheinendes Wasser 73.0 1417 100.0 siedendes Wasser 54.0 1049 11.75059952038369 temperierte Luft

By linear regressession, we calculate Amonton's zero temperature to -241°C

In [8]:
(defn ordinate [xs ys]
  (let [n    (count xs)
        s_x  (reduce + xs)
        s_y  (reduce + ys)
        xy   (map #(* %1 %2) xs ys)
        s_xy (reduce + xy)
        x2   (map #(* % %) xs)
        s_x2 (reduce + x2)
        sx_2 (* s_x s_x)]
    (/ (- (* s_y s_x2) (* s_x s_xy))
       (- (* n s_x2) sx_2))))

(apply ordinate (map (transpose (tb2 1417)) [0 2]))

-241.17904934486012

Page 47, §89: His boiling point is 1370, the old was 1417. "Siedepunkt Wasser 1370 bestimmt", "Ich werde als eine runde Zahl C = 1370 annehmen. Amontons fand 1417"

In [9]:
(mapv #(round % 2)
      (map #(apply ordinate
                   (map (transpose (tb2 %)) [0 2]))
           [1360 1370 1380 1417]))

-279.37 -271.82 -264.66 -241.18

So, based on his raw data, Lambert's zero temperature is -271,82°C. It is the extrapolation based on his 1370 degrees. If we assume a measurement error of +-10 digits, Lambert's value is bound between -265°C and -280°C, indeed an improvement to Amonton.